In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import pandas as pd
import glob
import h5py
import netCDF4 as nc
from pytz import all_timezones

In [2]:
def MDJtoDT(mdj2k):
    """Take the modified julian day from 2000, convert to seconds, add to seconds since jan 1,00:00:00.00"""
    t0 = dt.datetime.utcfromtimestamp(0.0)
    t2k = dt.datetime(2000,1,1,0,0,0)
    t2k = (t2k-t0).total_seconds()
    tmdj = mdj2k * 86400
    date = dt.datetime.utcfromtimestamp(t2k+tmdj)
    return date

In [3]:
ds = nc.Dataset('test.hdf')

In [20]:
ds["DATETIME"]

<class 'netCDF4._netCDF4.Variable'>
>f8 DATETIME(fakeDim6)
    VAR_NAME: DATETIME
    VAR_DESCRIPTION: datetime (UT), modified Julianday
    VAR_NOTES: defined relative to reference datetime of Jan. 1, 2000 at 0:00:00 UT which is equal to 0.00
    VAR_SIZE: 4640
    VAR_DEPEND: DATETIME
    VAR_DATA_TYPE: DOUBLE
    VAR_UNITS: MJD2K
    VAR_SI_CONVERSION: 0.0;86400.0;s
    VAR_VALID_MIN: -5000.0
    VAR_VALID_MAX: 8000.0
    VAR_FILL_VALUE: -90000.0
unlimited dimensions: 
current shape = (4640,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [22]:
ds["O3.MIXING.RATIO.VOLUME_ABSORPTION.SOLAR"]

<class 'netCDF4._netCDF4.Variable'>
>f4 O3.MIXING.RATIO.VOLUME_ABSORPTION.SOLAR(fakeDim23, fakeDim24)
    VAR_NAME: O3.MIXING.RATIO.VOLUME_ABSORPTION.SOLAR
    VAR_DESCRIPTION: retrieved vertical profile of   XXXXfrom solar absorption measurements, in VMR units
    VAR_NOTES: retrieval algorithm PROFFIT96; HITRAN  spectroscopy
    VAR_SIZE: 4640;41
    VAR_DEPEND: DATETIME;ALTITUDE
    VAR_DATA_TYPE: REAL
    VAR_UNITS: ppmv
    VAR_SI_CONVERSION: 0.0;1.0E-6;1
    VAR_VALID_MIN: -1.0
    VAR_VALID_MAX: 100.0
    VAR_FILL_VALUE: -90000.0
unlimited dimensions: 
current shape = (4640, 41)
filling on, default _FillValue of 9.969209968386869e+36 used

In [23]:
ds.close()